In [14]:
import pandas as pd
import numpy as np
from datetime import datetime
import re
import csv

In [15]:
#locating the dataset
pd.set_option('display.max_columns', None)

df_trustpilot = pd.read_csv('data/data_trustpilot_first.csv', engine='python')

df_trustpilot.head()


,username,number_reviews,location,rating,text,date_of_experience,date_posted,subject,answer,company,verification,answered
0,Rob Crane,2,CA,5,The company rep I worked with made my transact...,2024-06-15,2024-10-23 04:17:44,The company rep I worked with made my…,0,Flashbay,0,0
1,Pat Anderson,1,US,5,I highly recommend using Flashbay. Immediately...,2024-10-16,2024-10-16 19:34:05,I highly recommend using Flashbay,0,Flashbay,1,0
2,Margarita Orlova,1,CZ,5,I had the pleasure of working with Shelby Gibs...,2024-10-10,2024-10-17 10:27:44,Great customer service,0,Flashbay,1,0
3,Paola Rivas,1,US,5,I had a fantastic experience with Brian Truong...,2024-10-21,2024-10-21 22:38:50,Outstanding Support and Attentive Service,0,Flashbay,1,0
4,Fiona Mckelvey Keenan,3,CA,5,My number-one go-to for computer accessories. ...,2024-07-11,2024-10-23 04:09:05,My number-one go-to for computer…,0,Flashbay,0,0


In [16]:
#convert date_posted and date_of_experience to datetime

df_trustpilot['date_posted'] = pd.to_datetime(df_trustpilot['date_posted'])
df_trustpilot['date_of_experience'] = pd.to_datetime(df_trustpilot['date_of_experience'])

#Fixing the date_posted column
#The date_posted is from the CET timezone, however, there are countries in the list that are in a different timezone.
#That way, if someone posts at 9 pm in New York City, it appears as 3 am CET, potentially skewing the data.
#We need to fix this column so that date_posted represents the original date and time posted, from the view of the user writing the review.
#Maps the timezone difference from the view of the CET timezone
timezone_offsets = {
    # Western Europe (CET/CEST)
    'DE': 0, 'FR': 0, 'IT': 0, 'ES': 0, 'NL': 0, 'BE': 0, 'AT': 0, 'CH': 0, 'SE': 0,
    'DK': 0, 'NO': 0, 'PL': 0, 'CZ': 0, 'PT': 0, 'HU': 0, 'SK': 0, 'SI': 0, 'LU': 0,
    'MT': 0, 'HR': 0, 'BA': 0, 'AL': 0, 'ME': 0, 'RS': 0, 'MK': 0, 'AD': 0, 'LI': 0,
    
    # Eastern Europe
    'FI': 1, 'EE': 1, 'LV': 1, 'LT': 1, 'UA': 1, 'RO': 1, 'BG': 1, 'GR': 1, 'BY': 1,
    'MD': 1, 'TR': 1, 'CY': 1,
    
    # UK and related territories
    'GB': -1, 'IE': -1, 'IM': -1, 'JE': -1, 'GG': -1, 'GI': -1,
    
    # Russia and Central Asia
    'RU': 3, 'KZ': 4, 'UZ': 5, 'KG': 5,
    
    # North America and Greenland
    'US': -7, 'CA': -7, 'MX': -7, 'PR': -6, 'VI': -6, 'GU': 10, 'GL': -3,  # Added Greenland
    
    # Central America and Caribbean
    'GT': -8, 'HN': -8, 'SV': -8, 'NI': -8, 'CR': -8, 'PA': -7,
    'BS': -7, 'CU': -7, 'JM': -7, 'DO': -6, 'BB': -6, 'TT': -6,
    'KN': -6, 'AG': -6, 'VC': -6, 'AI': -6, 'BM': -6, 'KY': -6,
    'TC': -6, 'AW': -6, 'CW': -6, 'BQ': -6, 'SX': -6, 'GP': -6,
    
    # South America
    'BR': -5, 'AR': -5, 'CL': -6, 'CO': -7, 'VE': -6, 'PE': -7,
    'EC': -7, 'BO': -6, 'PY': -6, 'UY': -5,
    
    # East Asia
    'CN': 7, 'JP': 8, 'KR': 8, 'TW': 7, 'MN': 7, 'HK': 7, 'MO': 7,
    
    # Southeast Asia
    'SG': 7, 'MY': 7, 'ID': 7, 'TH': 7, 'VN': 7, 'PH': 7,
    'MM': 6.5, 'KH': 7, 'BN': 7,
    
    # South Asia
    'IN': 4.5, 'PK': 4, 'BD': 5, 'NP': 4.75, 'LK': 4.5, 'MV': 4,
    
    # Central Asia and Middle East
    'AE': 3, 'SA': 2, 'IL': 2, 'IR': 2.5, 'IQ': 2, 'OM': 3,
    'KW': 2, 'QA': 2, 'BH': 2, 'JO': 2, 'LB': 2, 'AF': 3.5,
    'AM': 3, 'AZ': 3, 'GE': 3,
    
    # Oceania
    'AU': 9, 'NZ': 11, 'FJ': 11, 'PF': 10, 'MP': 9,
    
    # Africa
    'ZA': 1, 'EG': 1, 'MA': 0, 'DZ': 0, 'TN': 0, 'LY': 1,
    'NG': 0, 'GH': 0, 'KE': 2, 'UG': 2, 'RW': 1, 'CD': 1,
    'GA': 0, 'CM': 0, 'ZW': 1, 'CI': 0, 'SS': 2,  # Added South Sudan
    'RE': 3,  # Added Réunion
    
    # European Islands and Territories
    'IS': -1, 'FO': -1, 'AX': 1,
    
    # Disputed or Special Territories
    'XK': 0  # Kosovo (using CET)
}

#Checks whether there are countries missing in our dictionary that are present in our df
countries_in_df = df_trustpilot['location'].unique()
countries_in_mapping = set(timezone_offsets.keys())
missing_countries = set(countries_in_df) - countries_in_mapping
if len(missing_countries) > 0:
    print("Countries in dataset but not in timezone mapping:")
    print(missing_countries)
    print(f"\nTotal missing countries: {len(missing_countries)}")
else:
    print("All countries in the dataset are covered in our timezone mapping!")

def adjust_to_local_time(df_trustpilot):
    """
    Adjusts timestamps from CET to local time based on country codes
    Creates local_hour and local_date_posted columns while preserving original date_posted
    
    Parameters:
    df_trustpilot: DataFrame with 'date_posted' and 'location' columns
    
    Returns:
    DataFrame with new 'local_hour' and 'local_date_posted' columns
    """
    # Initialize new columns
    df_trustpilot['local_hour'] = None
    df_trustpilot['local_date_posted'] = None
    
    # Process non-CET countries
    for country_code, offset in timezone_offsets.items():
        # Get mask for current country
        mask = df_trustpilot['location'] == country_code
        
        # Adjust time for matching rows
        if offset != 0:  # Skip CET countries
            local_datetime = pd.to_datetime(df_trustpilot.loc[mask, 'date_posted']) + pd.Timedelta(hours=offset)
            df_trustpilot.loc[mask, 'local_date_posted'] = local_datetime.dt.date
            df_trustpilot.loc[mask, 'local_hour'] = local_datetime.dt.hour
    
    # Process CET countries (offset = 0)
    cet_mask = df_trustpilot['location'].isin([k for k, v in timezone_offsets.items() if v == 0])
    cet_datetime = pd.to_datetime(df_trustpilot.loc[cet_mask, 'date_posted'])
    df_trustpilot.loc[cet_mask, 'local_date_posted'] = cet_datetime.dt.date
    df_trustpilot.loc[cet_mask, 'local_hour'] = cet_datetime.dt.hour
    
    # Evaluation steps
    def evaluate_timezone_adjustment():
        # 1. Check for missing values
        null_local_hour = df_trustpilot['local_hour'].isnull().sum()
        null_local_date = df_trustpilot['local_date_posted'].isnull().sum()
        
        # 2. Check for unexpected hours (should be 0-23)
        invalid_hours = df_trustpilot[~df_trustpilot['local_hour'].between(0, 23)].shape[0]
        
        # 3. Check date changes
        date_changes = (df_trustpilot['local_date_posted'] != 
                       pd.to_datetime(df_trustpilot['date_posted']).dt.date).sum()
        
        print("\nEvaluation Results:")
        print(f"Missing local hours: {null_local_hour}")
        print(f"Missing local dates: {null_local_date}")
        print(f"Invalid hours (not 0-23): {invalid_hours}")
        print(f"Number of date changes due to timezone adjustment: {date_changes}")
        
        # Sample of significant changes
        if date_changes > 0:
            print("\nSample of records where date changed:")
            date_change_mask = (df_trustpilot['local_date_posted'] != 
                              pd.to_datetime(df_trustpilot['date_posted']).dt.date)
            sample_changes = df_trustpilot[date_change_mask].sample(min(5, date_changes))
            print(sample_changes[['location', 'date_posted', 'local_date_posted', 'local_hour']].to_string())
    
    # Run evaluation
    evaluate_timezone_adjustment()
    
    return df_trustpilot

# Apply the function and overwrite the existing dataframe
df_trustpilot = adjust_to_local_time(df_trustpilot)

df_trustpilot['local_hour'] = df_trustpilot['local_hour'].astype('int')
df_trustpilot['local_date_posted'] = pd.to_datetime(df_trustpilot['local_date_posted'])


All countries in the dataset are covered in our timezone mapping!

Evaluation Results:
Missing local hours: 0
Missing local dates: 0
Invalid hours (not 0-23): 0
Number of date changes due to timezone adjustment: 9807

Sample of records where date changed:
      location         date_posted local_date_posted local_hour
58531       US 2024-02-09 00:27:56        2024-02-08         17
7264        US 2017-06-29 04:45:26        2017-06-28         21
75054       US 2021-12-15 02:39:18        2021-12-14         19
75177       US 2021-12-15 02:17:55        2021-12-14         19
60823       US 2024-01-08 00:30:21        2024-01-07         17


In [17]:
df_trustpilot['days_between_experience_and_post'] = (df_trustpilot['local_date_posted'] - df_trustpilot['date_of_experience']).dt.days

# Extract day of week (Monday=0, Sunday=6)
df_trustpilot['day_of_week_posted'] = df_trustpilot['local_date_posted'].dt.dayofweek

In [18]:
df_trustpilot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78404 entries, 0 to 78403
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   username                          78404 non-null  object        
 1   number_reviews                    78404 non-null  int64         
 2   location                          78404 non-null  object        
 3   rating                            78404 non-null  int64         
 4   text                              78404 non-null  object        
 5   date_of_experience                78404 non-null  datetime64[ns]
 6   date_posted                       78404 non-null  datetime64[ns]
 7   subject                           78404 non-null  object        
 8   answer                            78404 non-null  object        
 9   company                           78404 non-null  object        
 10  verification                      78404 non-nu

In [19]:
#Split set into numeric, categorical and datetime variables
#Provide statistical metrics for numeric variables

numeric_variables = df_trustpilot.select_dtypes(include = ['int', 'float'])
categorical_variables = df_trustpilot.select_dtypes(include=['object']) 
datetime_variables = df_trustpilot.select_dtypes(include=['datetime64'])

print("Numeric variables are: ", numeric_variables.columns)
print("Categorical variables are: ", categorical_variables.columns)
print("Datetime variables are: ", datetime_variables.columns)

print(numeric_variables.describe())


Numeric variables are:  Index(['number_reviews', 'rating', 'verification', 'answered', 'local_hour',
       'days_between_experience_and_post', 'day_of_week_posted'],
      dtype='object')
Categorical variables are:  Index(['username', 'location', 'text', 'subject', 'answer', 'company'], dtype='object')
Datetime variables are:  Index(['date_of_experience', 'date_posted', 'local_date_posted'], dtype='object')
       number_reviews        rating  verification      answered    local_hour  \
count    78404.000000  78404.000000  78404.000000  78404.000000  78404.000000   
mean         3.751186      3.520662      0.645426      0.248393     12.380567   
std          9.933424      1.697901      0.478387      0.432084      4.898129   
min          1.000000      1.000000      0.000000      0.000000      0.000000   
25%          1.000000      1.000000      0.000000      0.000000      9.000000   
50%          1.000000      4.000000      1.000000      0.000000     12.000000   
75%          3.000000

In [20]:
#Upon review, it appears that the number_reviews, rating, hour_posted, day_of_week_posted and verification columns are fine and do not include faulty data. 
#However, the minimum value for the days_between_experience_and_post column is -1, which should not appear.
#We will further review days_between_experience_and_post.

negative_days = df_trustpilot[df_trustpilot['days_between_experience_and_post'] < 0]

print(negative_days)
print(len(negative_days))

#There is only one entry with negative days, which will be deleted

df_trustpilot = df_trustpilot[df_trustpilot['days_between_experience_and_post'] >= 0]

negative_days_after = df_trustpilot[df_trustpilot['days_between_experience_and_post'] < 0]
print(len(negative_days_after))


                   username  number_reviews location  rating  \
1412     Messinger Mortuary               1       US       5   
1427      Christopher Jacky               1       US       5   
1431           Scott Osborn               1       US       5   
1446   Estefani Carrasquedo               1       US       5   
1447             Wendy Goff               1       US       5   
...                     ...             ...      ...     ...   
78390      Richard  Lemieux               2       US       3   
78391         Harvey Outlaw               1       US       3   
78392                   Pat               1       US       1   
78393           John Mangan               1       US       3   
78394        Quicken Member               1       US       3   

                                                    text date_of_experience  \
1412   We are very pleased with the flash drives we o...         2022-09-08   
1427   Reps were very helpful and polite. Product was...         2022-09-

In [21]:
#Check outcome of removing negative days
numeric_variables = df_trustpilot.select_dtypes(include = ['int', 'float'])
print("Minimum value for days_between_experience_and_post:")
print(df_trustpilot['days_between_experience_and_post'].min())

print("Summary statistics for numeric variables in the dataset:")
print(numeric_variables.describe())

Minimum value for days_between_experience_and_post:
0
Summary statistics for numeric variables in the dataset:
       number_reviews        rating  verification      answered    local_hour  \
count    73001.000000  73001.000000  73001.000000  73001.000000  73001.000000   
mean         3.881522      3.495363      0.660991      0.252079     11.877837   
std         10.242879      1.706886      0.473376      0.434209      4.676573   
min          1.000000      1.000000      0.000000      0.000000      0.000000   
25%          1.000000      1.000000      0.000000      0.000000      8.000000   
50%          1.000000      4.000000      1.000000      0.000000     11.000000   
75%          3.000000      5.000000      1.000000      1.000000     15.000000   
max        592.000000      5.000000      1.000000      1.000000     23.000000   

       days_between_experience_and_post  day_of_week_posted  
count                      73001.000000        73001.000000  
mean                           3.71

In [22]:
#Calculating mode, mean and median for each numeric variable

for variable in numeric_variables:
    
    mode_value = df_trustpilot[variable].mode()[0]
    mode_count = df_trustpilot[variable].value_counts().iloc[0]
    
    mean_value = df_trustpilot[variable].mean()
    median_value = df_trustpilot[variable].median()
    
    print(f"\nAnalysis for {variable}:")
    if variable == 'rating':
        print(f"The most common rating is {mode_value}, appearing {mode_count} times")
        print(f"The mean rating is: {mean_value:.2f}")
        print(f"The median rating is: {median_value:.2f}")
    elif variable == 'number_reviews':
        print(f"The most common number of reviews is {mode_value}, appearing {mode_count} times")
        print(f"The mean number of reviews is: {mean_value:.2f}")
        print(f"The median rating is: {median_value:.2f}")
    elif variable == 'hour_posted':
        print(f"The most common hour of posting is {mode_value}, appearing {mode_count} times")
        print(f"The mean hour of posting is: {mean_value:.2f}")
        print(f"The median rating is: {median_value:.2f}")
    elif variable == 'day_of_week_posted':
        print(f"The most common day of posting is {mode_value}, appearing {mode_count} times")
        print(f"The mean day of posting is: {mean_value:.2f}")
        print(f"The median rating is: {median_value:.2f}")
    elif variable == 'days_between_experience_and_post':
        print(f"The most common number of days between experience and post is {mode_value}, appearing {mode_count} times")
        print(f"The mean number of days between experience and post is: {mean_value:.2f}")
        print(f"The median rating is: {median_value:.2f}")
    elif variable == 'verification':
        print(f"The most common verification status is {mode_value}, appearing {mode_count} times")
        print(f"The mean verification value is: {mean_value:.2f}")
        print(f"The median rating is: {median_value:.2f}")


Analysis for number_reviews:
The most common number of reviews is 1, appearing 40527 times
The mean number of reviews is: 3.88
The median rating is: 1.00

Analysis for rating:
The most common rating is 5, appearing 35876 times
The mean rating is: 3.50
The median rating is: 4.00

Analysis for verification:
The most common verification status is 1, appearing 48253 times
The mean verification value is: 0.66
The median rating is: 1.00

Analysis for answered:

Analysis for local_hour:

Analysis for days_between_experience_and_post:
The most common number of days between experience and post is 0, appearing 57461 times
The mean number of days between experience and post is: 3.72
The median rating is: 0.00

Analysis for day_of_week_posted:
The most common day of posting is 2, appearing 13595 times
The mean day of posting is: 2.66
The median rating is: 3.00


In [23]:
#Calculate distribution of categorical variables

print("Categorical Variables Distribution:")
for col in categorical_variables.columns:
    if col not in ['text', 'subject']:  
        print(f"\n{col} distribution:")
        print(df_trustpilot[col].value_counts(normalize=True).head())

print("Day of Week Distribution:")
print(df_trustpilot['day_of_week_posted'].value_counts(normalize=True))


Categorical Variables Distribution:

username distribution:
username
customer          0.005520
Customer          0.004192
John              0.002452
Quicken Member    0.002000
David             0.002000
Name: proportion, dtype: float64

location distribution:
location
US    0.651594
GB    0.235490
CA    0.019260
DE    0.011931
FR    0.006726
Name: proportion, dtype: float64

answer distribution:
answer
0                                                                                                                                                                                  0.747921
Fantastic! We are glad you had an excellent experience working with us.Thank you for trusting Flashbay and we look forward to working with you again soon. -Flashbay Management    0.000973
Fantastic! Thank you for trusting Flashbay with your USB needs. We look forward to working with you next time!-Flashbay Management                                                 0.000603
We're glad to hear Quicken is

In [24]:
#Calculate earliest and latest dates in dataset

print("Temporal variables distribution:")
for col in datetime_variables.columns:
    print(f"\n{col} range:")
    print(f"Earliest: {df_trustpilot[col].min()}")
    print(f"Latest: {df_trustpilot[col].max()}")

Temporal variables distribution:

date_of_experience range:
Earliest: 2009-06-09 00:00:00
Latest: 2024-10-22 00:00:00

date_posted range:
Earliest: 2009-06-09 08:57:17
Latest: 2024-10-23 10:16:57

local_date_posted range:
Earliest: 2009-06-09 00:00:00
Latest: 2024-10-23 00:00:00


In [25]:
import os

os.makedirs('data', exist_ok=True)

csv_path = 'data/data_trustpilot.csv'
df_trustpilot.to_csv(csv_path, index=False)
print(f"DataFrame exported to {csv_path}")

DataFrame exported to data/data_trustpilot.csv
